### Install a package to connect to a MySQL database

In [1]:
%pip install mysql-connector-python


  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/9d/1b/a7b17532d6c7972e9173d9cc6fad5f94b949bcaaafa744f4e68a255ebd37/mysql_connector_python-8.1.0-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
     -- ---------------------------------- 30.7/527.0 kB 660.6 kB/s eta 0:00:01
     ---------- --------------------------- 143.4/527.0 kB 2.1 MB/s eta 0:00:01
     -------------------------------------  522.2/527.0 kB 5.4 MB/s eta 0:00:01
     -------------------------------------- 527.0/527.0 kB 4.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.9 MB 16.0 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/10.9 MB 15.4 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/10.9 MB 13.1 MB/s eta 0:00:01
   --- ------------------------------------ 1.0/10.9 MB 13.1 MB/

In [77]:
# Main query
OCT_2021_QUERY = """
SELECT
  q.Id AS id,
  q.Title AS title,
  q.Body AS body,
  q.AcceptedAnswerId AS accepted_answer_id,
  q.ViewCount AS view_count,
  q.Tags AS tags,
  q.AnswerCount AS answer_count,
  q.Score AS question_score,
  q.CreationDate AS creation_date,
  a.Score AS answer_score,
  a.Body AS stackoverflow_answer
FROM
  Posts q
LEFT JOIN
  Posts a
ON
  q.AcceptedAnswerId = a.Id
WHERE
  q.AnswerCount > 0
  AND q.AcceptedAnswerId > 0
  AND q.CreationDate >= '2021/10/01'
  AND a.Score >= 0
"""

# Query for testing (much smaller size)
AUG_2023_QUERY = """
SELECT
  q.Id AS id,
  q.Title AS title,
  q.Body AS body,
  q.AcceptedAnswerId AS accepted_answer_id,
  q.ViewCount AS view_count,
  q.Tags AS tags,
  q.AnswerCount AS answer_count,
  q.Score AS question_score,
  q.CreationDate AS creation_date,
  a.Score AS answer_score,
  a.Body AS stackoverflow_answer
FROM
  Posts q
LEFT JOIN
  Posts a
ON
  q.AcceptedAnswerId = a.Id
WHERE
  q.AnswerCount > 0
  AND q.AcceptedAnswerId > 0
  AND q.CreationDate >= '2023/08/01' AND q.CreationDate <= '2023/08/31'
  AND a.Score >= 0
"""

# Query for testing (much smaller size)
AUG_2023_QUERY_NO_DATE = """
SELECT
  q.Id AS id,
  q.Title AS title,
  q.Body AS body,
  q.AcceptedAnswerId AS accepted_answer_id,
  q.ViewCount AS view_count,
  q.Tags AS tags,
  q.AnswerCount AS answer_count,
  q.Score AS question_score,
  a.Score AS answer_score,
  a.Body AS stackoverflow_answer
FROM
  Posts q
LEFT JOIN
  Posts a
ON
  q.AcceptedAnswerId = a.Id
WHERE
  q.AnswerCount > 0
  AND q.AcceptedAnswerId > 0
  AND q.CreationDate >= '2023/08/01' AND q.CreationDate <= '2023/08/31'
  AND a.Score >= 0
"""

In [78]:
import mysql.connector
import pandas as pd

# Connect to the local database
so_db = mysql.connector.connect(host="localhost",
                                port=3006,      # Default port might be 3036
                                database='stackoverflow_com',
                                user="root",
                                passwd="n311.?R0Npe{Sneqz=`}xM7yrm,Hl4|J<4]4`cwEdF!.*t0~X_")

# Run main query
try:
    results = pd.read_sql(OCT_2021_QUERY, so_db)
    so_db.close()
except Exception as e:
    so_db.close()
    print(str(e))

C:\Users\Mark\AppData\Local\Temp\ipykernel_35124\598071335.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  results = pd.read_sql(OCT_2021_QUERY, so_db)


In [80]:
# Check query results
print(results.shape)
results.head()

(887296, 11)


,id,title,body,accepted_answer_id,view_count,tags,answer_count,question_score,creation_date,answer_score,stackoverflow_answer
0,69399929,How to Allow Multiple Exports from a Vue3 Sing...,<p>I'm making a Vue3 Single File Component for...,69410743,1085,<javascript><vue.js><es6-modules>,2,1,2021-10-01 00:02:03,2,<p>I was able to kinda get this to work by not...
1,69399930,Xlookup and return value not from the same row,<p>Is it possible to use XLOOKUP lookup and re...,69488440,770,<excel><excel-formula>,3,0,2021-10-01 00:02:18,0,"<p><a href=""https://i.stack.imgur.com/bkthw.pn..."
2,69399940,How to publish a crate with optional dependency?,<p>I have some trouble about publishing crates...,69401269,318,<rust><rust-cargo>,1,2,2021-10-01 00:04:09,1,<p><code>cargo publish</code> is the correct w...
3,69399941,What is best practice for instantiating a vari...,<p>I frequently find myself in situations wher...,69399963,71,<python><loops><scope>,2,-1,2021-10-01 00:04:10,1,<p>I wouldn't write the first assignment since...
4,69399943,Is there a way to automatically position and f...,<h1>The problem</h1>&#xA;<p>When I generate a ...,69487376,887,<r><r-markdown><flextable>,1,0,2021-10-01 00:04:20,3,<p>You can achieve that by using &quot;officed...


In [81]:
# Export results to CSV
results.to_csv("data_dump_dataset.csv", index=False)

### Demo: differences between CSV export and pickle (pure object representation) export

In [53]:
# Connect to local database
so_db = mysql.connector.connect(host="localhost",
                                port=3006,      # Default port might be 3036
                                database='stackoverflow_com',
                                user="root",
                                passwd="n311.?R0Npe{Sneqz=`}xM7yrm,Hl4|J<4]4`cwEdF!.*t0~X_")

# Run test queries
try:
    aug_results = pd.read_sql(AUG_2023_QUERY, so_db)
    aug_results_no_date = pd.read_sql(AUG_2023_QUERY_NO_DATE, so_db)
    so_db.close()
except Exception as e:
    so_db.close()
    print(str(e))

C:\Users\Mark\AppData\Local\Temp\ipykernel_35124\2604944330.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aug_results = pd.read_sql(AUG_QUERY, so_db)


MySQL Connection not available.


C:\Users\Mark\AppData\Local\Temp\ipykernel_35124\2604944330.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aug_results_no_date = pd.read_sql(AUG_QUERY_NO_DATE, so_db)


In [54]:
# Preview test query with dates
aug_results.head()

,id,title,body,accepted_answer_id,view_count,tags,answer_count,question_score,creation_date,answer_score,stackoverflow_answer
0,76807756,Azure App Service unable to deploy Streamlit app,<p>I have a Streamlit app I am trying to deply...,76836102,174,<python><azure><azure-web-app-service><streamlit>,1,0,2023-08-01 00:01:47,1,<p>This error is happening while importing <st...
1,76807759,I need to remove white spacing for lines that ...,"<p><a href=""https://i.stack.imgur.com/Sndaa.pn...",76808152,52,<linux><awk><sed><txt>,3,2,2023-08-01 00:02:02,2,<p>Assumptions:</p>&#xA;<ul>&#xA;<li>The input...
2,76807774,Strapi Graphql Inconsistent 401 Authorization ...,<p>Hey guys I've been developing a site using ...,76809588,30,<graphql><cors><authorization><apollo><strapi>,1,0,2023-08-01 00:07:21,0,<p>Hey guys I think I found the culprit. It's ...
3,76807776,Can I avoid unnest() skipping unset array elem...,<p>I had expected <code>unnest()</code> to ret...,76807856,36,<arrays><postgresql><sparse-matrix><unnest>,1,1,2023-08-01 00:07:44,2,"<p>Not sure why it is <a href=""https://www.pos..."
4,76807780,How to set up Angular service workers?,<p>I'm trying to set up Service Workers in my ...,76826527,66,<angular><angular-cli><progressive-web-apps><a...,1,0,2023-08-01 00:10:16,0,<p>As it seems you were using very old version...


In [57]:
# Preview test query without dates
aug_results_no_date.head()

,id,title,body,accepted_answer_id,view_count,tags,answer_count,question_score,answer_score,stackoverflow_answer
0,76807756,Azure App Service unable to deploy Streamlit app,<p>I have a Streamlit app I am trying to deply...,76836102,174,<python><azure><azure-web-app-service><streamlit>,1,0,1,<p>This error is happening while importing <st...
1,76807759,I need to remove white spacing for lines that ...,"<p><a href=""https://i.stack.imgur.com/Sndaa.pn...",76808152,52,<linux><awk><sed><txt>,3,2,2,<p>Assumptions:</p>&#xA;<ul>&#xA;<li>The input...
2,76807774,Strapi Graphql Inconsistent 401 Authorization ...,<p>Hey guys I've been developing a site using ...,76809588,30,<graphql><cors><authorization><apollo><strapi>,1,0,0,<p>Hey guys I think I found the culprit. It's ...
3,76807776,Can I avoid unnest() skipping unset array elem...,<p>I had expected <code>unnest()</code> to ret...,76807856,36,<arrays><postgresql><sparse-matrix><unnest>,1,1,2,"<p>Not sure why it is <a href=""https://www.pos..."
4,76807780,How to set up Angular service workers?,<p>I'm trying to set up Service Workers in my ...,76826527,66,<angular><angular-cli><progressive-web-apps><a...,1,0,0,<p>As it seems you were using very old version...


In [2]:
import os
from pathlib import Path

# Construct paths
cwd = Path().absolute()
ec_path          = os.path.join(cwd, "export_comparison")

path_pkl         = os.path.join(ec_path, "test_pickle.pkl")
path_pkl_no_date = os.path.join(ec_path, "test_pickle_no_date.pkl")

path_csv         = os.path.join(ec_path, "test_csv.csv")
path_csv_no_date = os.path.join(ec_path, "test_csv_no_date.csv")

# Export each dataset to pickle and CSV for optional later testing
aug_results.to_pickle("test_pickle.pkl")
aug_results.to_csv("test_csv.csv", index=False)

aug_results_no_date.to_pickle("test_pickle_no_date.pkl")
aug_results_no_date.to_csv("test_csv_no_date.csv", index=False)

# Import the files we just saved into dataframes
df_pkl = pd.read_pickle(path_pkl)
df_csv = pd.read_csv(path_csv)

df_pkl_no_date = pd.read_pickle(path_pkl_no_date)
df_csv_no_date = pd.read_csv(path_csv_no_date)

In [3]:
# Quick evaluation of the with-date dataframes
df_pkl.equals(df_csv)

False

In [4]:
# Quick evaluation of the without-date dataframes
df_pkl_no_date.equals(df_csv_no_date)

True

In [5]:
from pandas.testing import assert_frame_equal

# More in-depth evaluation of the test dataframes
#   Errors in cell output show differences; no differences results in no output
assert_frame_equal(df_pkl_no_date, df_csv_no_date, check_dtype=False)
assert_frame_equal(df_pkl, df_csv, check_dtype=False)

AssertionError: DataFrame.iloc[:, 8] (column name="creation_date") are different

DataFrame.iloc[:, 8] (column name="creation_date") values are different (100.0 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
[left]:  <DatetimeArray>
['2023-08-01 00:01:47', '2023-08-01 00:02:02', '2023-08-01 00:07:21',
 '2023-08-01 00:07:44', '2023-08-01 00:10:16', '2023-08-01 00:11:59',
 '2023-08-01 00:13:05', '2023-08-01 00:13:11', '2023-08-01 00:14:07',
 '2023-08-01 00:15:45',
 ...
 '2023-08-30 23:34:09', '2023-08-30 23:39:50', '2023-08-30 23:39:58',
 '2023-08-30 23:42:26', '2023-08-30 23:46:13', '2023-08-30 23:47:10',
 '2023-08-30 23:48:23', '20

In [6]:
import numpy as np

df_csv_fix = df_csv.astype({"creation_date": "datetime64[ns]"})

In [7]:
from pandas.testing import assert_frame_equal

# Compare the dataframes again after the fix is applied
#   Errors in cell output show differences
#   No differences results in no output
assert_frame_equal(df_pkl_no_date, df_csv_no_date, check_dtype=False)
assert_frame_equal(df_pkl, df_csv_fix)

So if desiring a strictly identical import from CSV as what the dataframe originally was, a conversion of the data type can be made for the creation_date column.